In [5]:
using JuMP
using Plots
gr()

Plots.GRBackend()

In [6]:
include("polynomials.jl")

PF

In [7]:
JuMP.getvalue{N, T, S}(p::SP.SPolynomial{N, T, S}) = SP.SPolynomial(Val{S}(), getvalue.(p.coeffs))

In [8]:
function require_continuity!(model, piecewise::PF.PiecewiseFunction)
    t = piecewise.breaks
    for i = 2:(length(t) - 1)
        @constraint model PF.from_below(piecewise, t[i]) .== PF.from_above(piecewise, t[i])
    end
end

require_continuity! (generic function with 1 method)

In [9]:
function piecewise_polynomial_variable!(model, domain, dimension::Integer, degree::Integer)
    C = @variable(model, [i=1:dimension, j=1:(degree + 1), k=1:(length(domain) - 1)], basename="C")
    polys = [SP.SPolynomial(Val{:t}(), tuple([C[:,j,k] for j in 1:(degree + 1)]...)) for k in 1:(length(domain) - 1)]
    p = PF.PiecewiseFunction(domain, polys)
end

piecewise_polynomial_variable! (generic function with 1 method)

In [46]:
using JuMP

horizon = 20
dt = 0.1
t = 0:dt:(horizon * dt)
mass = 1
g = 9.81
dim = 1

model = Model()

qcom = piecewise_polynomial_variable!(model, t, dim, 2)
vcom = SP.derivative.(qcom)
acom = SP.derivative.(vcom)
require_continuity!(model, qcom)
require_continuity!(model, vcom)

qlimb = piecewise_polynomial_variable!(model, t, dim, 1)
vlimb = SP.derivative.(qlimb)
require_continuity!(model, qlimb)

@variable model force[1:dim, 1:horizon] >= 0

@constraint model [i=1:dim, j=1:horizon] acom(t[j] + dt / 2)[i] == force[i, j] - mass * g

# Kinematics
@constraint model [j=1:horizon] 0.1 <= qcom(t[j])[1] - qlimb(t[j])[1]
@constraint model [j=1:horizon] qcom(t[j])[1] - qlimb(t[j])[1] <= 0.2

@variable model contact[1:horizon] Bin
@constraint model [i=1:dim, j=1:horizon] force[i, j] <= 100 * contact[j]
@constraint model [i=1:dim, j=1:horizon] PF.from_below(qlimb, t[j + 1])[i] <= 10 * (1 - contact[j])
@constraint model [i=1:dim, j=1:horizon] qlimb(t[j])[i] >= 0


@constraints model begin
    qcom(0) .== 1
    vcom(0) .== 0
    qlimb(0) .== 0.85
    vlimb(0) .== 0
end

@constraint model PF.from_below(qcom, t[end])[1] == 0.15
posture_error = sum((qcom(t[j])[1] - qlimb(t[j])[1] - 0.15)^2 for j in 1:horizon)
@objective model Min sum(force.^2) + 0.1 * posture_error

model
solve(model)

Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Optimize a model with 182 rows, 140 columns and 437 nonzeros
Model has 80 quadratic objective terms
Variable types: 120 continuous, 20 integer (20 binary)
Coefficient statistics:
  Matrix range     [1e-02, 1e+02]
  Objective range  [3e-02, 3e-02]
  QObjective range [2e-01, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e-01, 1e+01]
Presolve removed 110 rows and 81 columns
Presolve time: 0.00s
Presolved: 72 rows, 59 columns, 191 nonzeros
Presolved model has 57 quadratic objective terms
Variable types: 47 continuous, 12 integer (12 binary)

Root relaxation: objective 5.299687e+03, 104 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 5299.68721    0   11          - 5299.68721      -     -    0s
H    0     0                    10799.385232 5299.68721  50.9%   

:Optimal

In [47]:
q = getvalue.(qcom)
v = getvalue.(vcom)
ql = getvalue.(qlimb)
a = getvalue.(acom)
z = getvalue.(contact)
f = getvalue.(force)

1×20 Array{Float64,2}:
 0.0  0.0  0.0  0.0  65.25  0.0  0.0  …  9.88968  9.80433  7.32125  2.44043

In [48]:
tspan = 0:0.001:(horizon*dt - 0.001)
plt = plot([x -> q(x)[1], x -> v(x)[1], x -> a(x)[1]], tspan, layout=(3,1))
plot!(plt, x -> ql(x)[1], tspan)
plt

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
 
 1.5 
 
 
 0.0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 1.0 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 30.6037,3.93701 30.8865,3.93757 31.1694,3.93925 31.4523,3.94206 31.7352,3.94598 32.018,3.95103 32.3009,3.9572 32.5838,3.96449 32.8666,3.9729 33.1495,3.98244 
 33.4324,3.9931 33.7153,4.00487 33.9981,4.01777 34.281,4.0318 34.5639,4.04694 34.8467,4.06321 35.1296,4.08059 35.4125,4.0991 35.6954,4.11873 35.9782,4.13948 
 36.2611,4.16136 36.544,4.18436 36.8268,4.20847 37.1097,4.23371 37.3926,4.26007 37.6755,4.28756 37.9583,4.31616 38.2412,4.34589 38.5241,4.37674 38.8069,4.40871 
 39.0898,4.4418 39.3727,4.47601 39.6555,4.51135 39.9384,4.5478 40.2213,4.58538 40.5042,4.62408 40.787,4.66391 41.0699,4.70485 41.3528,4.74692 41.6356,4.7901 
 41.9185,4.83441 42.2014,4.87984 42.4843,4.9264 42.7671,4.97407 43.05,5.02287 43.3329,5.07279 43.6157,5.12383 43.8986,5.17599 44.1815,5.22927 44.4644,5.28368 
 44.7472,5.3392 45.0301,5.39585 45.313,5.45362 45.5958,5.51251 45.8787,5.57253 46.1616,5.63366 46.4445,5.69592 46.7273,5.7593 47.0102,5.8238 47.2931,5.88942 
 47.5759,5.95617 47.8588,6.02403 48.1417,6.09302 48.4246,6.16313 48.7074,6.23436 48.9903,6.30672 49.2732,6.38019 49.556,6.45479 49.8389,6.53051 50.1218,6.60735 
 50.4047,6.68531 50.6875,6.76439 50.9704,6.8446 51.2533,6.92593 51.5361,7.00838 51.819,7.09195 52.1019,7.17664 52.3847,7.26245 52.6676,7.34939 52.9505,7.43745 
 53.2334,7.52663 53.5162,7.61693 53.7991,7.70835 54.082,7.8009 54.3648,7.89456 54.6477,7.98935 54.9306,8.08526 55.2135,8.18229 55.4963,8.28045 55.7792,8.37972 
 56.0621,8.48012 56.3449,8.58164 56.6278,8.68428 56.9107,8.78804 57.1936,8.89293 57.4764,8.99893 57.7593,9.10606 58.0422,9.21431 58.325,9.32368 58.6079,9.43417 
 58.8908,9.54579 59.1737,9.65852 59.4565,9.77238 59.7394,9.88736 60.0223,10.0035 60.3051,10.1207 60.588,10.239 60.8709,10.3585 61.1538,10.4791 61.4366,10.6008 
 61.7195,10.7236 62.0024,10.8476 62.2852,10.9727 62.5681,11.0989 62.851,11.2262 63.1339,11.3546 63.4167,11.4842 63.6996,11.6149 63.9825,11.7467 64.2653,11.8796 
 64.5482,12.0137 64.8311,12.1488 65.1139,12.2851 65.3968,12.4225 65.6797,12.5611 65.9626,12.7007 66.2454,12.8415 66.5283,12.9834 66.8112,13.1264 67.094,13.2706 
 67.3769,13.4158 67.6598,13.5622 67.9427,13.7097 68.2255,13.8584 68.5084,14.0081 68.7913,14.159 69.0741,14.311 69.357,14.4641 69.6399,14.6184 69.9228,14.7737 
 70.2056,14.9302 70.4885,15.0878 70.7714,15.2466 71.0542,15.4064 71.3371,15.5674 71.62,15.7295 71.9029,15.8927 72.1857,16.057 72.4686,16.2225 72.7515,16.3891 
 73.0343,16.5568 73.3172,16.7256 73.6001,16.8955 73.883,17.0666 74.1658,17.2388 74.4487,17.4121 74.7316,17.5865 75.0144,17.7621 75.2973,17.9388 75.5802,18.1166 
 75.863,18.2955 76.1459,18.4755 76.4288,18.6567 76.7117,18.839 76.9945,19.0224 77.2774,19.2069 77.5603,19.3926 77.8431,19.5793 78.126,19.7672 78.4089,19.9562 
 78.6918,20.1464 78.9746,20.3376 79.2575,20.53 79.5404,20.7235 79.8232,20.9181 80.1061,21.1139 80.389,21.3108 80.6719,21.5088 80.9547,21.7079 81.2376,21.9081 
 81.5205,22.1095 81.8033,22.3119 82.0862,22.5155 82.3691,22.7202 82.652,22.9261 82.9348,23.1331 83.2177,23.3411 83.5006,23.5503 83.7834,23.7607 84.0663,23.9721 
 84.3492,24.1847 84.6321,24.3984 84.9149,24.6132 85.1978,24.8292 85.4807,25.0462 85.7635,25.2644 86.0464,25.4837 86.3293,25.7041 86.6122,25.9257 86.895,26.1483 
 87.1779,26.3721 87.4608,26.597 87.7436,26.8231 88.0265,27.0502 88.3094,27.2785 88.5922,27.5079 88.8751,27.7384 89.158,27.9701 89.4409,28.2028 89.7237,28.4367 
 90.0066,28.6717 90.2895,28.9079 90.5723,29.1451 90.8552,29.3835 91.1381,29.623 91.421,29.8636 91.7038,30.1053 91.9867,30.3482 92.2696,30.5922 92.5524,30.8373 
 92.8353,31.0835 93.1182,31.3308 93.4011,31.5793 93.6839,31.8289 93.9668,32.0796 94.2497,32.3315 94.5325,32.5844 94.8154,32.8385 95.0983,33.0937 95.381